In [1]:
import requests

import pandas as pd
import json

In [2]:
data = pd.read_csv('分时数据/上海疫情（地址）.csv')

In [3]:
add = data['发布位置'].tolist()

In [4]:
district=[]
lngitude=[]
latitude=[]
formattted_address=[]

In [5]:
for address in add:
    #根据百度地图api接口获取正地址编码也就是经纬度
    url1='http://api.map.baidu.com/geocoding/v3/?address='+address+'&output=json&ak=9dwaEGuGuXDmD5QSOENXvwI7Q0QGg40a&callback=showLocation'

    #获取经纬度
    resp1=requests.get(url1)
    resp1_str=resp1.text
    resp1_str=resp1_str.replace('showLocation&&showLocation','')
    resp1_str=resp1_str[1:-1]
    resp1_json=json.loads(resp1_str)
    if resp1_json.get('status')==1:
        formattted_address.append(None)
        lngitude.append(None)
        latitude.append(None)
        district.append(None)
    else:
        location=resp1_json.get('result').get('location')

        #根据经纬度获取结构化地址
        lng=location.get('lng')
        lat=location.get('lat')
        url2='http://api.map.baidu.com/reverse_geocoding/v3/?ak=9dwaEGuGuXDmD5QSOENXvwI7Q0QGg40a&output=json&coordtype=wgs84ll&location='+str(lat)+','+str(lng)+''
        resp2=requests.get(url2)

        resp2_json=json.loads(resp2.text)
        #提取结构化地址
        formattted_address.append(resp2_json.get('result').get('formatted_address'))
        lngitude.append(resp1_json.get('result').get('location').get('lng'))
        latitude.append(resp1_json.get('result').get('location').get('lat'))
        district.append(resp2_json.get('result').get('addressComponent').get('district'))

In [6]:
data['district']=district
data['lngitude']=lngitude
data['latitude']=latitude
data['formattted_address']=formattted_address

In [7]:
data.head()

,id,bid,user_id,用户昵称,微博正文,头条文章url,发布位置,艾特用户,话题,转发数,...,点赞数,发布时间,发布工具,微博图片url,微博视频url,retweet_id,district,lngitude,latitude,formattted_address
0,4.760000e+15,LqzaOgKS8,3685548934,Tina娜娜512,很开心，同小区邻居“拼多多”20人成团，79元菜套餐；又可以撑过一段时间了……#上海疫情##团菜#,NaN,上海·遵义小区,NaN,"上海疫情,团菜",0,...,6,2022/4/28 7:53 PM,iPhone客户端,['https://wx3.sinaimg.cn/large/dbad0386ly1h1po...,NaN,NaN,长宁区,121.415542,31.219316,上海市长宁区定西路1310弄4号1504室
1,4.760000e+15,Lqws181zo,5348374653,那我就叫榆烟吧,继团购理发师之后，我们小区有了开始团购月嫂的需求…#上海#这疫情得难为多少人啊…,NaN,上海·紫竹半岛,NaN,上海,0,...,0,2022/4/28 12:57 PM,元宇宙的HarmonyOS设备,['https://wx1.sinaimg.cn/large/005PXezjly1h1pc...,NaN,NaN,闵行区,121.472217,31.032018,上海市闵行区通海路
2,4.754840e+15,Ln0CCdUb0,7344998815,夏天1314526,不是通知我们5号解封吗？怎么又来7＋7上海什么时候能解封了啊，吃不上了都疫情L夏天13145...,NaN,上海·紫薇茗庭,NaN,NaN,0,...,0,2022/4/5 10:25 AM,疫情超话,NaN,http://f.video.weibocdn.com/o0/uptwAPnElx07V27...,NaN,松江区,121.254771,31.097383,上海市松江区洞伟路
3,4.754680e+15,LmWvEAgEt,5625306290,一起去遨游太空吧,#隔离日记##上海疫情#到底是谁想出来让我们在没有医护的隔离点隔离啊？半夜哭着给我朋友打电话...,NaN,上海·梓潼会所,NaN,"隔离日记,上海疫情",5,...,69,2022/4/4 11:57 PM,iPhone客户端,['https://wx3.sinaimg.cn/large/0068Hd8Cly1h0y4...,NaN,NaN,浦东新区,121.568055,31.118489,上海市浦东新区周浦
4,4.755030e+15,Ln5DlqD8Z,5625306290,一起去遨游太空吧,#上海疫情##隔离日记#隔离D3的中饭和晚饭，今天发了一个口罩。自己带的八宝粥加餐，病友们救...,NaN,上海·梓潼会所,NaN,"上海疫情,隔离日记",0,...,0,2022/4/5 11:10 PM,iPhone客户端,['https://wx2.sinaimg.cn/large/0068Hd8Cly1h0z8...,NaN,NaN,浦东新区,121.568055,31.118489,上海市浦东新区周浦


pandas 删除空值
目的： 将该含有过多空值的行/列删除

`DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)`

axis:维度，axis=0表示index行,axis=1表示columns列，默认为0

how:"all"表示这一行或列中的元素全部缺失（为nan）才删除这一行或列，"any"表示这一行或列中只要有元素缺失，就删除这一行或列

thresh： axis中至少有thresh个非缺失值，否则删除。（对此参数存在疑问的，我会在另外的文章写到）

subset：在某些列的子集中选择出现了缺失值的列删除，不在子集中的含有缺失值得列或行不会删除（有axis决定是行还是列）

inplace：刷选过缺失值得新数据是存为副本还是直接在原数据上进行修改。默认是False，即创建新的对象进行修改，原对象不变，和深复制和浅复制有些类似。

In [8]:
data.to_csv('分时数据/上海疫情（结构化地址2）.csv',encoding='utf-8_sig')